I am going to put this into a SQL database, I am just more comfortable with manipulating data using SQL

In [1]:
import pandas as pd
import sqlite3 as sl

In [2]:
#create database object
conn = sl.connect('covid.db')
c = conn.cursor()

here are the column names that need to be created in our table
'id', 
'sex', 
'patient_type', 
'entry_date', 
'date_symptoms', 
'date_died',
'intubed', 
'pneumonia', 
'age', 
'pregnancy', 
'diabetes', 
'copd',
'asthma', 
'inmsupr', 
'hypertension', 
'other_disease', 
'cardiovascular',
'obesity', 
'renal_chronic', 
'tobacco', 
'contact_other_covid',
'covid_res', 
'icu'

In [3]:
# #create table and populate
# covid = pd.read_csv('covid.csv', index_col=0, encoding='latin-1')

# c.execute('DROP TABLE covid')
# c.execute('CREATE TABLE covid (id int, sex int, patient_type int, entry_date datetime, date_symptoms datetime, date_died datetime, intubed int, pneumonia int, age int, pregnancy int, diabetes int, copd int, asthma int,inmsupr int, hypertension int, other_disease int, cardiovascular int, obesity int, renal_chronic int, tobacco int, contact_other_covid int, covid_res int, icu int)')
# conn.commit()

# covid.to_sql('covid',conn, if_exists='replace')

In [4]:
df = pd.read_sql(''' select * from covid limit 10;''',conn)
df

,id,sex,patient_type,entry_date,date_symptoms,date_died,intubed,pneumonia,age,pregnancy,...,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
0,16169f,2,1,04-05-2020,02-05-2020,9999-99-99,97,2,27,97,...,2,2,2,2,2,2,2,2,1,97
1,1009bf,2,1,19-03-2020,17-03-2020,9999-99-99,97,2,24,97,...,2,2,2,2,2,2,2,99,1,97
2,167386,1,2,06-04-2020,01-04-2020,9999-99-99,2,2,54,2,...,2,2,2,2,1,2,2,99,1,2
3,0b5948,2,2,17-04-2020,10-04-2020,9999-99-99,2,1,30,97,...,2,2,2,2,2,2,2,99,1,2
4,0d01b5,1,2,13-04-2020,13-04-2020,22-04-2020,2,2,60,2,...,2,1,2,1,2,2,2,99,1,2
5,1beec8,2,2,16-04-2020,16-04-2020,29-04-2020,2,1,47,97,...,2,2,2,2,2,2,2,99,1,1
6,1.75E+56,2,2,22-04-2020,13-04-2020,9999-99-99,2,2,63,97,...,2,1,2,2,2,2,2,99,1,2
7,0ce1f5,1,1,21-05-2020,18-05-2020,9999-99-99,97,2,56,2,...,2,1,2,2,2,1,1,1,1,97
8,12b57a,1,1,22-04-2020,18-04-2020,9999-99-99,97,2,41,2,...,2,2,2,2,2,2,2,99,1,97
9,006b91,1,2,23-04-2020,18-04-2020,9999-99-99,1,1,39,2,...,2,2,2,2,1,2,2,99,1,2


In [5]:
def data_summary(df):
    eda_df= {}
    eda_df['null_sum'] = df.isnull().sum()
    eda_df['dtypes'] = df.dtypes
    eda_df['count'] = df.count()
    eda_df['mean'] = df.mean()
    eda_df['median'] = df.median()
    eda_df['max'] = df.max()
    eda_df['min'] = df.min()
    
    return pd.DataFrame(eda_df)

data_summary(df)

,null_sum,dtypes,count,mean,median,max,min
age,0,int64,10,44.1,44.0,63,24
asthma,0,int64,10,2.0,2.0,2,2
cardiovascular,0,int64,10,1.9,2.0,2,1
contact_other_covid,0,int64,10,79.5,99.0,99,1
copd,0,int64,10,2.0,2.0,2,2
covid_res,0,int64,10,1.0,1.0,1,1
date_died,0,object,10,NaN,NaN,9999-99-99,22-04-2020
date_symptoms,0,object,10,NaN,NaN,18-05-2020,01-04-2020
diabetes,0,int64,10,1.8,2.0,2,1
entry_date,0,object,10,NaN,NaN,23-04-2020,04-05-2020


In [6]:
df.head()

,id,sex,patient_type,entry_date,date_symptoms,date_died,intubed,pneumonia,age,pregnancy,...,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
0,16169f,2,1,04-05-2020,02-05-2020,9999-99-99,97,2,27,97,...,2,2,2,2,2,2,2,2,1,97
1,1009bf,2,1,19-03-2020,17-03-2020,9999-99-99,97,2,24,97,...,2,2,2,2,2,2,2,99,1,97
2,167386,1,2,06-04-2020,01-04-2020,9999-99-99,2,2,54,2,...,2,2,2,2,1,2,2,99,1,2
3,0b5948,2,2,17-04-2020,10-04-2020,9999-99-99,2,1,30,97,...,2,2,2,2,2,2,2,99,1,2
4,0d01b5,1,2,13-04-2020,13-04-2020,22-04-2020,2,2,60,2,...,2,1,2,1,2,2,2,99,1,2
